### Dart Open Api 사용해서 크롤링

In [86]:
import pandas as pd
from bs4 import BeautifulSoup
import webbrowser
import json
import requests

In [87]:
# 개발 참조 https://opendart.fss.or.kr/guide/main.do?apiGrpCd=DS003

In [88]:
# 요청값
# 내 API key니까 다들 하나씩 발급받으면 좋을듯!
# 1일 1만회 요청건수 제한
auth_key="847436c641de81093f023d156e8686df4e226471" 

# stock_code랑은 다른 company 고유코드. 
# 따오는거 정리해야할듯
crp_code ="00356370" 

# 사업연도 설정
사업연도 ="2018"

# 1분기보고서 : 11013
# 반기보고서 : 11012
# 3분기보고서 : 11014
# 사업보고서 : 11011
report_code = "11011"

In [91]:
url = "https://opendart.fss.or.kr/api/fnlttSinglAcnt.json?crtfc_key={0}&corp_code={1}&bsns_year={2}&reprt_code={3}"
url = url.format(auth_key, crpcode, 사업연도, report_code)

In [92]:
response = requests.get(url)
data = json.loads(response.content)

In [93]:
data

{'status': '000',
 'message': '정상',
 'list': [{'rcept_no': '11011',
   'corp_code': '00126380',
   'stock_code': '005930',
   'account_nm': '유동자산',
   'fs_div': 'CFS',
   'fs_nm': '연결재무제표',
   'sj_div': 'BS',
   'sj_nm': '재무상태표',
   'thstrm_nm': '제 50 기',
   'thstrm_dt': '2018.12.31 현재',
   'thstrm_amount': '174,697,424,000,000',
   'frmtrm_nm': '제 49 기',
   'frmtrm_dt': '2017.12.31 현재',
   'frmtrm_amount': '146,982,464,000,000',
   'bfefrmtrm_nm': '제 48 기',
   'bfefrmtrm_dt': '2016.12.31 현재',
   'bfefrmtrm_amount': '141,429,704,000,000',
   'ord': '1'},
  {'rcept_no': '11011',
   'corp_code': '00126380',
   'stock_code': '005930',
   'account_nm': '비유동자산',
   'fs_div': 'CFS',
   'fs_nm': '연결재무제표',
   'sj_div': 'BS',
   'sj_nm': '재무상태표',
   'thstrm_nm': '제 50 기',
   'thstrm_dt': '2018.12.31 현재',
   'thstrm_amount': '164,659,820,000,000',
   'frmtrm_nm': '제 49 기',
   'frmtrm_dt': '2017.12.31 현재',
   'frmtrm_amount': '154,769,626,000,000',
   'bfefrmtrm_nm': '제 48 기',
   'bfefrmtrm_dt': 

In [84]:
data["list"][1]

{'rcept_no': '11011',
 'corp_code': '00126380',
 'stock_code': '005930',
 'account_nm': '비유동자산',
 'fs_div': 'CFS',
 'fs_nm': '연결재무제표',
 'sj_div': 'BS',
 'sj_nm': '재무상태표',
 'thstrm_nm': '제 50 기',
 'thstrm_dt': '2018.12.31 현재',
 'thstrm_amount': '164,659,820,000,000',
 'frmtrm_nm': '제 49 기',
 'frmtrm_dt': '2017.12.31 현재',
 'frmtrm_amount': '154,769,626,000,000',
 'bfefrmtrm_nm': '제 48 기',
 'bfefrmtrm_dt': '2016.12.31 현재',
 'bfefrmtrm_amount': '120,744,620,000,000',
 'ord': '3'}